# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    
    # create empty graph and open file
    G = nx.DiGraph()
    file = open(infile)
    
    # for confirming correctness
    total_nodes = 0;
    total_edges = 0;
    total=0;
    
    # read and parse each line in file. Update graph
    for line in file:
        line = line.strip().split()
        if len(line) > 0:
            if line[0] == 'p':               # handle problem statement
                total_nodes = int(line[2])
                total_edges = int(line[3])
            elif line[0] == 'n':             # handle nodes
                G.add_node(line[1], demand = int(line[2]))                
            elif line[0] == 'a':             # handle edges                
                temp = str(total_nodes + 1)  # a way to name new nodes without conflict
                
                G.add_edge(line[1], temp, weight = int(line[5]), capacity = int(line[4]))
                G.add_edge(temp, line[2], capacity = int(line[4]))
                
                # for reduction we end up with one additional node and edge for each edge                
                total_nodes += 1
                total_edges += 1
    
    # checks for correct number of nodes and edges, not necessary for reduction
    if G.number_of_nodes() != total_nodes:
        print "ERROR: Nodes in graph do not match nodes in file"
        
    if G.number_of_edges() != total_edges:
        print "ERROR: Edges in graph do not match edges in file"
    
    return G

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')


print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    
    # will hold our data for variables in LP
    nodes = []
    edges = []
    nodeProps = {}
    edgeProps = {}
        
    # demand variables: extract nodes and demands from G
    for n in G.nodes():
        nodes.append(n)
        if 'demand' in G.node[n]:
            nodeProps[n] = G.node[n]['demand']
        else: 
            nodeProps[n] = 0 # demand 0 if not set
        
    # weight and capacity variables: extract edges, weights and capacities from G
    for (u, v) in G.edges():
        edges.append((u, v))
        if 'weight' in G.edge[u][v]:
            edgeProps[(u, v)] = [G.edge[u][v]['weight'], G.edge[u][v]['capacity']]  
        else:
            edgeProps[(u, v)] = [0, G.edge[u][v]['capacity']]
        
    # flow variables
    flows = pulp.LpVariable.dicts("Flow", edges, None, None, pulp.LpInteger)

    # problem statement     
    prob = pulp.LpProblem("Minimum Cost Flow", pulp.LpMinimize)
 
    # objective function
    prob += pulp.lpSum([flows[e]* edgeProps[e][0] for e in edges]) # sum of flows * weights
    
    # capacity constraints
    for e in edges:
        flows[e].bounds(0, edgeProps[e][1]) # lower bound (0), upper bound (capacity)        
    
    # flow conservation constraints
    for n in nodes:
        f_in = pulp.lpSum([flows[(u,v)] for (u,v) in edges if v == n]) # sum of in_degree flow
        f_out = pulp.lpSum([flows[(u,v)] for (u,v) in edges if u == n]) # sum of out_degree flow        
        prob += (f_out - f_in == nodeProps[n]) # f_out - f_in = demand
    
    # solve the linear program     
    prob.solve()
 
    # return value of objective function as integer
    return int(pulp.value(prob.objective))


The following will check that the LP finds the same optimal values as previously.

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
